# CCAPM Pricing Example

Last update: Nov 9th, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Consider the following example with $u\left(c\right)=\ln\left(c\right), \beta=0.96, y=\left[1.5,1,0.5\right]$ with a transition matrix
$$
\Pi=\left[\begin{array}{ccc}
0.5 & 0.25 & 0.25\\
0.25 & 0.5 & 0.25\\
0.25 & 0.25 & 0.5
\end{array}\right]
$$

In [2]:
n = 3
y_grid = np.array([1.5, 1, 0.5])
Pi = np.array([[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5]])
beta = 0.96

In [3]:
# Compute the SDF

m = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        m[i, j] = beta * y_grid[i] / y_grid[j] # the derivative of ln(x) is 1/x
        
# Print SDF
print(m)

[[0.96 1.44 2.88]
 [0.64 0.96 1.92]
 [0.32 0.48 0.96]]


In [4]:
m = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        m[i, j] = beta * ((y_grid[j] / y_grid[i]) ** -2) # the derivative of ln(x) is 1/x
        
# Print SDF
print(m)

[[0.96       2.16       8.64      ]
 [0.42666667 0.96       3.84      ]
 [0.10666667 0.24       0.96      ]]


In [5]:
# Start the iteration

T = 1000
P = np.zeros((T, n))

for i in range(T - 1):
    for j in range(n):
        P[i + 1, j] = np.dot(Pi[j, :], m[j, :] * (y_grid + P[i, :]))
    if np.max(np.abs(P[i + 1, :] - P[i, :])) < 1e-10:
        print(f"Convergence happens after {i} periods")
        break

# Print the prices
print(P[i + 1, :])

Convergence happens after 588 periods
[65.60526316 29.26315789  7.39473684]


Alternatively, we can solve the equation system to see whether our solution is correct.

In [6]:
coeff = np.eye(n) - Pi * m
RHS = np.dot(Pi * m, y_grid)
np.linalg.solve(coeff, RHS)

array([65.60526316, 29.26315789,  7.39473684])

Hence it seems that our solution is correct.

Now we wrap it into a function and carry out some comparative statics.

In [11]:
def asset_price(beta, y_grid, Pi, gamma):
    n = len(y_grid)
    m = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            m[i, j] = (y_grid[j] / y_grid[i]) ** (-gamma)
    
    T = 100000
    P = np.zeros((T, n))

    for i in range(T - 1):
        for j in range(n):
            P[i + 1, j] = beta * np.dot(Pi[j, :], m[j, :] * (y_grid + P[i, :]))
        if np.max(np.abs(P[i + 1, :] - P[i, :])) < 1e-10:
            print(f"Convergence happens after {i} periods")
            break
    
    return P[i + 1, :], beta*m

In [13]:
P_grid = [asset_price(beta, y_grid, Pi, gamma)[0] for gamma in [1, 1.5, 2]]
m_grid = [asset_price(beta, y_grid, Pi, gamma)[1] for gamma in [1, 1.5, 2]]

Convergence happens after 573 periods
Convergence happens after 580 periods
Convergence happens after 588 periods
Convergence happens after 573 periods
Convergence happens after 580 periods
Convergence happens after 588 periods


In [14]:
P_grid

[array([36., 24., 12.]),
 array([47.33047514, 25.82139587,  9.17548836]),
 array([65.60526316, 29.26315789,  7.39473684])]

In [15]:
m_grid

[array([[0.96, 1.44, 2.88],
        [0.64, 0.96, 1.92],
        [0.32, 0.48, 0.96]]),
 array([[0.96      , 1.76363261, 4.98830633],
        [0.52255781, 0.96      , 2.71529004],
        [0.18475209, 0.33941125, 0.96      ]]),
 array([[0.96      , 2.16      , 8.64      ],
        [0.42666667, 0.96      , 3.84      ],
        [0.10666667, 0.24      , 0.96      ]])]

Let's first try a set of $y$ grid that is less volatile than the previous example.

In [9]:
Pi_new = np.array([[0.7, 0.15, 0.15], [0.15, 0.7, 0.15], [0.15, 0.15, 0.7]])

print(asset_price(beta, y_grid, Pi_new, 1))
print(asset_price(beta, y_grid, Pi_new, 2))

Convergence happens after 573 periods
(array([36., 24., 12.]), array([[1.        , 1.5       , 3.        ],
       [0.66666667, 1.        , 2.        ],
       [0.33333333, 0.5       , 1.        ]]))
Convergence happens after 588 periods
(array([64.60169491, 29.08474576,  7.55084746]), array([[1.        , 2.25      , 9.        ],
       [0.44444444, 1.        , 4.        ],
       [0.11111111, 0.25      , 1.        ]]))


In [10]:
y_grid_new = [1.8, 1.0, 0.2]

print(asset_price(beta, y_grid_new, Pi, 1))
print(asset_price(beta, y_grid_new, Pi, 2))

Convergence happens after 578 periods
(array([43.2, 24. ,  4.8]), array([[1.        , 1.8       , 9.        ],
       [0.55555556, 1.        , 5.        ],
       [0.11111111, 0.2       , 1.        ]]))
Convergence happens after 611 periods
(array([168.25263158,  52.07017544,   2.13333333]), array([[1.00000000e+00, 3.24000000e+00, 8.10000000e+01],
       [3.08641975e-01, 1.00000000e+00, 2.50000000e+01],
       [1.23456790e-02, 4.00000000e-02, 1.00000000e+00]]))
